# AS-AD model with floating exchange regime


**THE COMPLETE MODEL**

In this Data Project, we construct and simulate an AS-AD model with floating exchange regime. We import empirical evidence for Sweden. The empiricial evidence has been imported from Jordá-Schularick-Taylor Macrohistory Database. We observe data from 1994 to 2020, since Sweden adopted a floating exchange regime in 1993.

<em> Model inpsiration comes from "Introducing Advanced Macroeconomics - Growth and Business Cycles 2022"<em>

$$
\begin{align*}
AS:\pi_t&=\pi^e+\gamma(y_t-\bar{y})+s_t \\
AD: \pi_t&=\frac{\beta_1}{\hat{\beta_1}}e^r_{t-1}+\pi^e-\frac{1}{\hat{\beta_1}}(y_t-\bar{y})+\frac{1}{\hat{\beta_1}}z_t \\
RERD:  e^r_t&=e^r_{t-1}+\left(1+\frac{h}{\theta}\right)(\pi^f-\pi_t) \\
\pi^e &= \pi^f \\
\hat{\beta_1}&=\beta_1+\frac{h\beta_1}{\theta}+h\beta_2 \\
z_t&=\delta z_{t-1}+x_t, x_t \sim N(0,\sigma_x^2) \\
s_t&=\omega s_{t-1}+c_t, c_t \sim N(0,\sigma_c^2) \\
\end{align*}
$$

Where $\pi_t$ is inflation, $\pi^f$ is inflation abroad, $\pi^e$ is expectations to inflation, $y_t$ is output, $\bar{y}$ is structural output, $e^r_t$ is the real exchange, $z_t$ is demand-shock, $s_t$ is supply-shock, $h$ describes at what degree the centralbank wants to minimize deviations in inflation, $\theta$ is adjustment speed in nominal exchange rate expectation, when the nominal exchange rate deviates from the structural level, $\bar{e}$.

Following some tideous algebra, the model can be solved to:

$$
\begin{align*}
\hat{y}_{t} &= \frac{1}{1+\hat{\beta}_{1}\gamma}\left[\beta_1e^r_{t-1}-\hat{\beta}_1\omega s_{t-1}-\hat{\beta}_1 c_t+\delta z_{t-1}+x_t\right] \\
\hat{\pi}_{t} &= \frac{\gamma}{1+\hat{\beta}_{1}\gamma}\left[\beta_1e^r_{t-1}-\hat{\beta}_1\omega s_{t-1}-\hat{\beta}_1 c_t+\delta z_{t-1}+x_t\right]+\omega s_{t-1}+c_t \\
e^r_{t} &= e^r_{t-1}-\left(1+\frac{h}{\theta}\right)\hat{\pi}_t
\end{align*}
$$

Where we have used the following definitions of gaps, $\hat{y_t}=y_t-\bar{y}$ and $\hat{\pi_t}=\pi_t-\pi^f$.

The parameters of the model are:
$\gamma, \beta_1, \delta, \omega, \sigma^2_x, \sigma^2_c, \beta_2, h, \theta$.

Importing packages:

In [2]:
%load_ext autoreload
%autoreload 2

from types import SimpleNamespace
import numpy as np
from scipy import optimize
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We use the pandas module to import data, and calculate empirical moments:

In [5]:
df = pd.read_csv('EmpiricalData.csv')
print(df)

                      Year;Outputgap;Inflationsgap
1994;-0 02570666;0                           56917
1995;-0 013269621;0                        1030303
1996;-0 019519662;-0                      14400527
1997;-0 019355048;-0                      13412385
1998;-0 010022954;-0                      21976285
1999;0  005959139;-0                      14071146
2000;0  020457053;-0                       7812912
2001;0  005749023;0                       10632411
2002;-0 000701339;0                        7667984
2003;-0 009578759;0                        5362319
2004;0  005327334;-0                      15059289
2005;0  009954682;-0                      13741765
2006;0  033552271;-0                       1554677
2007;0  046240392;0                         997365
2008;0  019971542;0                       28089592
2009;-0 046164425;-0                      23952569
2010;-0 006622572;-0                       1554677
2011;0  008240427;0                       18208169
2012;-0 016291467;-0           